In [11]:
# Imports

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

from keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [12]:
# Loading data

RB_TRAIN_PATH = "resources/rb_maps"
#RB_TEST_PATH = "resources/dataset_test_gp_rb"
NB_TRAIN_PATH = "resources/nb_maps"
#NB_TEST_PATH = "resources/dataset_test_gp_nb"

def load_scores(path):
    with open(os.path.join(path, "scores.txt"), "r") as file:
        scores = [float(line.strip()) for line in file]
    return np.array(scores)

def load_maps(path, gen): 
    base_file = ''
    if gen == "rb":
        base_file = "generated_map{}.txt"
    elif gen == "nb":
        base_file = "generated_map{}.txt"
    
    maps = []
    for i in range(50):
        with open(os.path.join(path, base_file.format(i)), "r") as file:
            map_data = [list(map(int, line.strip().split())) for line in file]
            maps.append(map_data)
    
    return np.array(maps)

# Load data

def data_loading(gen):
    if gen == 'rb':
        X_train, y_train = load_maps(RB_TRAIN_PATH, "rb"), load_scores(RB_TRAIN_PATH)
        #X_test, y_test = load_maps(RB_TRAIN_PATH, "rb"), load_scores(RB_TRAIN_PATH)

        X_train = X_train / 3.0
        #X_test  = X_test / 3.0
        
    elif gen == 'nb':
        X_train, y_train = load_maps(NB_TRAIN_PATH, "nb"), load_scores(NB_TRAIN_PATH)
        #X_test, y_test = load_maps(NB_TRAIN_PATH, "nb"), load_scores(NB_TRAIN_PATH)

        X_train = X_train / 2.0 
        #X_test = X_test / 2.0 
    
    return X_train, y_train


X_train, y_train = data_loading('nb')

# Add channel dimension
X_train = np.expand_dims(X_train, axis=-1)
#X_test = np.expand_dims(X_test, axis=-1)

In [13]:
def create_model(input_shape):
    model = Sequential()
    
    # Convolutional Layer 1
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    
    # Convolutional Layer 2
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    
    # Flatten and Dense Layers
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))  # Output layer (regression task)
    
    return model

input_shape = (22, 40, 1)
model = create_model(input_shape)

In [14]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

model.save('cnn_model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('cnn_model.h5')

# Verify the model
loaded_model.summary()

Epoch 1/100
2/2 [==============================] - 1s 432ms/step - loss: 6.0735 - mean_absolute_error: 2.2891 - val_loss: 1.1380 - val_mean_absolute_error: 0.9240
Epoch 2/100
2/2 [==============================] - 0s 91ms/step - loss: 1.4396 - mean_absolute_error: 0.9891 - val_loss: 1.0684 - val_mean_absolute_error: 0.8944
Epoch 3/100
2/2 [==============================] - 0s 64ms/step - loss: 1.6667 - mean_absolute_error: 1.0980 - val_loss: 0.6991 - val_mean_absolute_error: 0.7171
Epoch 4/100
2/2 [==============================] - 0s 64ms/step - loss: 1.1238 - mean_absolute_error: 0.8610 - val_loss: 0.3678 - val_mean_absolute_error: 0.5341
Epoch 5/100
2/2 [==============================] - 0s 56ms/step - loss: 0.8657 - mean_absolute_error: 0.7069 - val_loss: 0.7176 - val_mean_absolute_error: 0.6495
Epoch 6/100
2/2 [==============================] - 0s 56ms/step - loss: 1.0815 - mean_absolute_error: 0.7950 - val_loss: 0.6509 - val_mean_absolute_error: 0.5983
Epoch 7/100
2/2 [==========

In [15]:
#loss, mae = model.evaluate(X_test, y_test)
#print(f"Validation Loss: {loss}, MAE: {mae}")

#predictions = model.predict(X_test)

#for i in range(10):
    #print(f'Predicted: {predictions[i][0]}, Actual: {y_test[i]}')